<a href="https://colab.research.google.com/github/Ashik-Ahammad/road-defect-detection-imageProcessing/blob/main/Image_processing_techniques_for_road_defect_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing NumPy Library









In [ ]:
import numpy as np

This cell creates the necessary directory for Kaggle API credentials, copies the kaggle.json file (which contains the API key), and sets the correct file permissions to ensure secure access to Kaggle datasets.

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


Downloading Pothole Detection Dataset from Kaggle



In [ ]:
!kaggle datasets download atulyakumar98/pothole-detection-dataset

Dataset URL: https://www.kaggle.com/datasets/atulyakumar98/pothole-detection-dataset
License(s): CC0-1.0
pothole-detection-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


Extracting Pothole Detection Dataset



In [ ]:
!unzip /content/pothole-detection-dataset.zip -d /content/pathhole_dataset/

Archive:  /content/pothole-detection-dataset.zip
replace /content/pathhole_dataset/normal/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import cv2

# Load the image
image_path = "/content/pathhole_dataset/potholes/10.jpg"  # Change this to your image path
image = cv2.imread(image_path)

# Check if the image is loaded
if image is not None:
    print("Image Shape:", image.shape)  # (Height, Width, Channels)
else:
    print("Error: Image not found or cannot be loaded.")


In [ ]:
from PIL import Image
import os
import matplotlib.pyplot as plt

def resize_image(input_image_path, output_image_path, size=(224, 224)):
    original_image = Image.open(input_image_path).convert("RGB")  # Convert to RGB
    resized_image = original_image.resize(size)
    resized_image.save(output_image_path, "JPEG")  # Ensure it's saved as JPEG

def resize_images_in_directory(input_directory, output_directory):
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    resize_compare_flag = True
    for class_name in ['normal', 'potholes']:
        input_folder = os.path.join(input_directory, class_name)
        output_folder = os.path.join(output_directory, class_name)
        os.makedirs(output_folder, exist_ok=True)

        for filename in os.listdir(input_folder):
            input_image_path = os.path.join(input_folder, filename)
            output_image_path = os.path.join(output_folder, os.path.splitext(filename)[0] + ".jpg")  # Ensure JPG format

            # Ensure the output directory exists
            os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

            # Check if the file is an image (simple filter)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                print(f"Resizing {input_image_path}...")
                resize_image(input_image_path, output_image_path)
                if resize_compare_flag:
                    compare_images(input_image_path, output_image_path)
                    resize_compare_flag = False

def compare_images(original_image_path, resized_image_path):
    original_image = Image.open(original_image_path)
    resized_image = Image.open(resized_image_path)
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(original_image)
    ax[0].set_title(f'Original Image\nDimensions: {original_image.size}')
    ax[0].axis('off')
    ax[1].imshow(resized_image)
    ax[1].set_title(f'Resized Image\nDimensions: {resized_image.size}')
    ax[1].axis('off')
    plt.show()

# Specify the paths
input_directory = '/content/pathhole_dataset'
output_directory = '/content/resize_dataset'

# Resize images in the specified directories
resize_images_in_directory(input_directory, output_directory)


In [ ]:
from PIL import Image, ImageEnhance
import os
import random
import matplotlib.pyplot as plt

def enhance_image(input_path, enhancement_type='contrast', level=1.2):
    try:
        img = Image.open(input_path)

        # Ensure image is in a compatible format
        if img.mode == 'RGBA':
            img = img.convert('RGB')

        if enhancement_type == 'contrast':
            enhancer = ImageEnhance.Contrast(img)
        elif enhancement_type == 'brightness':
            enhancer = ImageEnhance.Brightness(img)
        else:
            raise ValueError("Unsupported enhancement type. Choose 'contrast' or 'brightness'.")

        # Enhance image
        enhanced_img = enhancer.enhance(level)
        return enhanced_img

    except IOError:
        print("Error opening or processing the image.")
        return None

def save_image(image, output_path):
    if image:
        image.save(output_path, format='JPEG')

def display_images(original_img, enhanced_img, title1='Original Image', title2='Enhanced Image'):
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(original_img)
    axes[0].set_title(title1)
    axes[0].axis('off')

    axes[1].imshow(enhanced_img)
    axes[1].set_title(title2)
    axes[1].axis('off')

    plt.show()

def main(dataset_path, enhancement_type='contrast', level=1.2):
    try:
        # Select a random class directory
        class_name = random.choice(os.listdir(dataset_path))
        class_dir = os.path.join(dataset_path, class_name)

        # Select a random image file
        image_file = random.choice([f for f in os.listdir(class_dir) if f.endswith('.jpg') or f.endswith('.png')])
        input_image_path = os.path.join(class_dir, image_file)

        # Enhance the image
        enhanced_img = enhance_image(input_image_path, enhancement_type, level)

        # Load the original image
        original_img = Image.open(input_image_path)

        # Display both images
        display_images(original_img, enhanced_img)

    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
dataset_path = "/content/resize_dataset"
main(dataset_path, enhancement_type='contrast', level=1.2)


In [ ]:
from PIL import Image, ImageEnhance
import os

def enhance_and_save_image(input_path, output_path, level=1.2):
    try:
        img = Image.open(input_path)

        # Convert image to RGB mode if it has an alpha channel
        if img.mode == 'RGBA':
            img = img.convert('RGB')

        enhancer = ImageEnhance.Contrast(img)
        enhanced_img = enhancer.enhance(level)

        # Construct output filename with the contrast level
        base, ext = os.path.splitext(output_path)
        output_final_path = f"{base}_contrast_{level}{ext}"

        enhanced_img.save(output_final_path)
        print(f"Enhanced image saved to: {output_final_path}")

    except Exception as e:
        print(f"Failed to enhance image {input_path}: {str(e)}")

def enhance_dataset_contrast(dataset_path, output_path):
    for class_name in os.listdir(dataset_path):
        class_dir = os.path.join(dataset_path, class_name)
        output_class_dir = os.path.join(output_path, class_name)

        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)

        for filename in os.listdir(class_dir):
            if filename.lower().endswith((".jpg", ".png")):
                input_image_path = os.path.join(class_dir, filename)
                output_image_path = os.path.join(output_class_dir, filename)
                enhance_and_save_image(input_image_path, output_image_path)

# Usage example
dataset_path = "/content/resize_dataset"
output_path = "/content/constrast_dataset"
enhance_dataset_contrast(dataset_path, output_path)


In [ ]:
from PIL import Image
import os
def gamma_correction(input_image_path, output_image_path, gamma=1.0):
    image = Image.open(input_image_path)
    if image.mode != 'RGB':
        image = image.convert('RGB')

    gamma_corrected_image = image.point(lambda p: 255 * ((p / 255.0) ** gamma))

    gamma_corrected_image.save(output_image_path)

def apply_gamma_correction_in_directory(input_directory, output_directory, gamma=2.2):
    comparison_done = True

    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    for subdir, dirs, files in os.walk(input_directory):
        for file in files:
            input_image_path = os.path.join(subdir, file)
            relative_path = os.path.relpath(input_image_path, input_directory)
            output_image_path = os.path.join(output_directory, relative_path)

            os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                print(f"Applying gamma correction to {input_image_path} with gamma={gamma}...")
                gamma_correction(input_image_path, output_image_path, gamma)
                if comparison_done:
                    compare_images(input_image_path, output_image_path)
                    comparison_done = False

def compare_images(original_image_path, processed_image_path):
    original_image = Image.open(original_image_path)
    processed_image = Image.open(processed_image_path)

    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(original_image)
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    ax[1].imshow(processed_image)
    ax[1].set_title('Gamma Corrected Image')
    ax[1].axis('off')

    plt.show()

input_directory = '/content/constrast_dataset'
gamma_output_directory = '/content/gamma_correction'
apply_gamma_correction_in_directory(input_directory, gamma_output_directory, gamma=1.3)


In [ ]:
import imgaug.augmenters as iaa
import numpy as np
import os
from skimage import io
import random

def augment_images_to_target(input_directory, output_directory, augmentation_pipeline, target_per_class=1500):
    """
    Args:
      input_directory: Directory containing the original images.
      output_directory: Directory to save the augmented images.
      augmentation_pipeline: Augmentation pipeline to apply to images.
      target_per_class: Target number of images per class.
    """
    for class_subdir in next(os.walk(input_directory))[1]:
        class_dir_path = os.path.join(input_directory, class_subdir)
        output_class_dir = os.path.join(output_directory, class_subdir)

        if not os.path.exists(output_class_dir):
            os.makedirs(output_class_dir)

        image_files = [f for f in os.listdir(class_dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif'))]

        total_images_needed = target_per_class - len(image_files)
        if total_images_needed <= 0:
            continue

        # Randomly shuffle the image files
        random.shuffle(image_files)

        images_per_original = total_images_needed // len(image_files)

        for file in image_files:
            input_image_path = os.path.join(class_dir_path, file)
            base_filename = os.path.splitext(file)[0]

            image = io.imread(input_image_path)
            image_augmented = []

            for _ in range(images_per_original):
                image_aug = augmentation_pipeline(image=image)
                image_augmented.append(image_aug)

            for idx, aug_image in enumerate(image_augmented, 1):
                filename = f"{base_filename}_augmented_{idx}.png"
                io.imsave(os.path.join(output_class_dir, filename), aug_image)

        # Augment remaining images to meet the target
        remaining_images_needed = target_per_class - len(os.listdir(output_class_dir))
        for idx in range(remaining_images_needed):
            file = random.choice(image_files)
            input_image_path = os.path.join(class_dir_path, file)
            base_filename = os.path.splitext(file)[0]

            image = io.imread(input_image_path)
            image_aug = augmentation_pipeline(image=image)
            filename = f"{base_filename}_augmented_extra_{idx}.png"
            io.imsave(os.path.join(output_class_dir, filename), image_aug)

rotation_angles = [90, 180, 270]

augmentation_pipeline = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Affine(rotate=random.choice(rotation_angles)),
    iaa.Multiply((0.8, 1.2)),
    iaa.LinearContrast((0.8, 1.2))
])

input_directory = '/content/gamma_correction'
output_directory = '/content/Final_Augment_Image'

augment_images_to_target(input_directory, output_directory, augmentation_pipeline)

In [ ]:
import os
import pandas as pd

input_directory = '/content/pathhole_dataset'  # Update with your actual path
augmented_directory = '/content/Final_Augment_Image'  # Update with your actual path where augmented images are saved

def count_images_in_directory(directory):
    counts = {}
    for root, dirs, files in os.walk(directory):
        for dir_name in dirs:
            dir_path = os.path.join(root, dir_name)
            count = len([name for name in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, name))])
            counts[dir_name] = count
    return counts

# Count original and augmented images
original_counts = count_images_in_directory(input_directory)
augmented_counts = count_images_in_directory(augmented_directory)

# Create a DataFrame
df = pd.DataFrame(list(original_counts.items()), columns=['Class', 'Original Count'])
df['Augmented Count'] = df['Class'].map(augmented_counts)

# Display the DataFrame
df


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure that seaborn is styled to have a more appealing look
sns.set(style="whitegrid")

# Set the size of the figure
plt.figure(figsize=(10, 6))

# Plot the 'Original Count' and 'Augmented Count' for each class
# Use a different color for each by specifying the 'palette' parameter
sns.barplot(data=df.melt(id_vars='Class', value_vars=['Original Count', 'Augmented Count'], var_name='Type', value_name='Count'),
            x='Class', y='Count', hue='Type', palette='viridis')

# Add some helpful UI components
plt.xticks(rotation=45, ha='right')  # Rotate class labels for better readability
plt.xlabel('Class')  # X-axis label
plt.ylabel('Count')  # Y-axis label
plt.title('Original vs Augmented Image Counts per Class')  # Title of the plot

# Display the plot
plt.tight_layout()  # Adjust the layout to make room for the rotated x-axis labels
plt.show()


In [ ]:
import os
import shutil
import random
from tqdm import tqdm

def split_dataset(source_dir, train_dir, test_dir, val_dir, split_ratio=(0.8, 0.1, 0.1)):
    for class_name in os.listdir(source_dir):
        class_dir = os.path.join(source_dir, class_name)
        train_class_dir = os.path.join(train_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)

        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)

        files = os.listdir(class_dir)
        random.shuffle(files)

        num_files = len(files)
        train_split = int(num_files * split_ratio[0])
        test_split = int(num_files * split_ratio[1])

        train_files = files[:train_split]
        test_files = files[train_split:train_split + test_split]
        val_files = files[train_split + test_split:]

        for file in tqdm(train_files, desc=f"Copying {class_name} train files"):
            shutil.copy(os.path.join(class_dir, file), os.path.join(train_class_dir, file))

        for file in tqdm(test_files, desc=f"Copying {class_name} test files"):
            shutil.copy(os.path.join(class_dir, file), os.path.join(test_class_dir, file))

        for file in tqdm(val_files, desc=f"Copying {class_name} validation files"):
            shutil.copy(os.path.join(class_dir, file), os.path.join(val_class_dir, file))

# Define paths
source_dir = "/content/Final_Augment_Image"
train_dir = "/content/train"
test_dir = "/content/test"
val_dir = "/content/validation"

# Split dataset
split_dataset(source_dir, train_dir, test_dir, val_dir)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def evaluate_model_performance(model, history, test_generator):
    # Plot Training and Validation Accuracy and Loss
    def plot_training_history(history):
        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']
        loss = history.history['loss']
        val_loss = history.history['val_loss']
        epochs = range(1, len(acc) + 1)

        plt.figure(figsize=(14, 5))
        plt.subplot(1, 2, 1)
        plt.plot(epochs, acc, 'bo-', label='Training accuracy')
        plt.plot(epochs, val_acc, 'ro-', label='Validation accuracy')
        plt.title('Training and Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(epochs, loss, 'bo-', label='Training loss')
        plt.plot(epochs, val_loss, 'ro-', label='Validation loss')
        plt.title('Training and Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.tight_layout()
        plt.show()

    # Plot the overall confusion matrix
    def plot_confusion_matrix(cm, class_names):
        plt.figure(figsize=(10, 7))
        sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()

    # Plot the confusion matrix for a specific class
    def plot_class_confusion_matrix(cm, class_names, class_index):
        class_cm = np.array([[cm[class_index, class_index], np.sum(cm[class_index, :]) - cm[class_index, class_index]],
                             [np.sum(cm[:, class_index]) - cm[class_index, class_index], np.sum(cm) + cm[class_index, class_index] - np.sum(cm[class_index, :]) - np.sum(cm[:, class_index])]])
        plt.figure(figsize=(6, 5))
        sns.heatmap(class_cm, annot=True, fmt='g', cmap='Blues', cbar=False,
                    xticklabels=['True ' + class_names[class_index], 'False ' + class_names[class_index]],
                    yticklabels=['True ' + class_names[class_index], 'False ' + class_names[class_index]])
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title(f'Confusion Matrix for Class: {class_names[class_index]}')
        plt.show()

    # Plot misclassifications for each class
    def plot_misclassifications(misclassifications, class_labels):
        plt.figure(figsize=(10, 6))
        plt.bar(class_labels, misclassifications, color='red')
        plt.xlabel('Class Labels')
        plt.ylabel('Number of Misclassifications')
        plt.title('Misclassifications for Each Class')
        plt.xticks(rotation=45)  # Rotate class labels to improve readability
        plt.tight_layout()  # Adjust subplots to give some padding
        plt.show()

    # Execute the plotting functions
    plot_training_history(history)

    # Prepare the test_generator and predict
    test_generator.reset()
    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_generator.classes
    class_labels = list(test_generator.class_indices.keys())

    # Compute confusion matrix
    conf_matrix = confusion_matrix(true_classes, predicted_classes)
    plot_confusion_matrix(conf_matrix, class_labels)

    # Plot confusion matrix for each class
    for idx, class_name in enumerate(class_labels):
        plot_class_confusion_matrix(conf_matrix, class_labels, idx)

    # Calculate and plot misclassifications
    misclassifications = np.sum(conf_matrix, axis=1) - np.diag(conf_matrix)
    plot_misclassifications(misclassifications, class_labels)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix

def generate_classification_report(model, test_generator):
    # Reset the test generator
    test_generator.reset()

    # Generate predictions from the model
    predictions = model.predict(test_generator)
    predicted_classes = np.argmax(predictions, axis=1)

    # Retrieve the actual labels
    true_classes = test_generator.classes

    # Get class labels from the generator
    class_labels = list(test_generator.class_indices.keys())

    # Generate the classification report
    report = classification_report(true_classes, predicted_classes, target_names=class_labels, output_dict=True)
    report_df = pd.DataFrame(report).transpose()

    # Remove unnecessary rows from the DataFrame
    report_df.drop(index=['accuracy', 'macro avg', 'weighted avg'], inplace=True)

    # Compute the confusion matrix
    conf_matrix = confusion_matrix(true_classes, predicted_classes)

    # Calculate specificities for each class
    specificities = []
    for i in range(len(class_labels)):
        true_negatives = conf_matrix.sum() - conf_matrix[i,:].sum() - conf_matrix[:,i].sum() + conf_matrix[i,i]
        false_positives = conf_matrix[:,i].sum() - conf_matrix[i,i]
        spec = true_negatives / (true_negatives + false_positives)
        specificities.append(spec)

    # Add specificities to the report DataFrame
    report_df['specificity'] = specificities

    # Print or return the DataFrame
    print(report_df)  # You could also return report_df if you want to use the DataFrame elsewhere

# Usage example:
# generate_classification_report(model, test_generator)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for your dataset
train_path = '/content/train'
test_path = '/content/test'
val_path = '/content/validation'
modelsave_path = "/content/MobileNetV2_model.h5"

# Define data generators
batch_size = 32

# Define and configure the data generator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators for validation and testing
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the testing data
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Import MobileNetV2 model
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Build custom classification model on top of MobileNetV2
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# Freeze pretrained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


# Train the model
checkpoint = ModelCheckpoint(
    modelsave_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=10,
    callbacks=[checkpoint]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)
print("Test Accuracy:", test_acc)

# Save the final model
model.save(modelsave_path)


In [ ]:
generate_classification_report(model, test_generator)

In [ ]:
evaluate_model_performance(model, history, test_generator)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for your dataset
train_path = '/content/train'
test_path = '/content/test'
val_path = '/content/validation'
modelsave_path = "/content/DenseNet201_model.h5"

# Define data generators
batch_size = 32

# Define and configure the data generator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators for validation and testing
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the testing data
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Import DenseNet201 model
base_model = DenseNet201(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Build custom classification model on top of DenseNet201
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# Freeze pretrained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
checkpoint = ModelCheckpoint(
    modelsave_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=10,
    callbacks=[checkpoint]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)
print("Test Accuracy:", test_acc)

# Save the final model
model.save(modelsave_path)


In [ ]:
generate_classification_report(model, test_generator)

In [ ]:
evaluate_model_performance(model, history, test_generator)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths for your dataset
train_path = '/content/train'
test_path = '/content/test'
val_path = '/content/validation'
modelsave_path = "/content/Xception_model.h5"

# Define data generators
batch_size = 32

# Define and configure the data generator for training with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Data generators for validation and testing
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(299, 299),  # Xception requires input shape (299, 299)
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the validation data
val_generator = val_datagen.flow_from_directory(
    val_path,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Load the testing data
test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Import Xception model
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(299, 299, 3)
)

# Build custom classification model on top of Xception
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu'),
    Dense(512, activation='relu'),
    Dense(train_generator.num_classes, activation='softmax')
])

# Freeze pretrained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(
     optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
checkpoint = ModelCheckpoint(
    modelsave_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=10,
    callbacks=[checkpoint]
)

# Evaluate the model
test_loss, test_acc = model.evaluate(
    test_generator,
    steps=test_generator.samples // batch_size
)
print("Test Accuracy:", test_acc)

# Save the final model
model.save(modelsave_path)


In [ ]:
generate_classification_report(model, test_generator)

In [ ]:
evaluate_model_performance(model, history, test_generator)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Concatenate, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.applications import Xception, DenseNet201

# Define paths for your dataset
train_path = '/content/train'
test_path = '/content/test'
val_path = '/content/validation'
modelsave_path = "/content/hybrid_xception_dense.h5"

# Define data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=24,
    class_mode='categorical')

validation_generator = val_test_datagen.flow_from_directory(
    val_path,
    target_size=(224, 224),
    batch_size=24,
    class_mode='categorical',
    shuffle=False)

test_generator = val_test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=24,
    class_mode='categorical',
    shuffle=False)

# Define input tensor
input_tensor = Input(shape=(224, 224, 3))

# Load Xception and DenseNet201 models
base_model_xception = Xception(weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model_densenet201 = DenseNet201(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the layers
for layer in base_model_xception.layers:
    layer.trainable = False

for layer in base_model_densenet201.layers:
    layer.trainable = False

# Global average pooling and concatenate layers
xception_output = GlobalAveragePooling2D()(base_model_xception.output)
densenet201_output = GlobalAveragePooling2D()(base_model_densenet201.output)
concatenated = Concatenate()([xception_output, densenet201_output])
concatenated = BatchNormalization()(concatenated)

# Dense layers
x = Dense(1024, activation='relu')(concatenated)
x = Dropout(0.2)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Model compilation
model = Model(inputs=input_tensor, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint(modelsave_path, save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Model training
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[checkpoint, early_stop]
)

# Model evaluation
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}, Test loss: {test_loss}')


In [ ]:
generate_classification_report(model, test_generator)

In [ ]:
evaluate_model_performance(model, history, test_generator)

In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.imagenet_utils import decode_predictions

# Load the trained model
model_path = "/content/DenseNet201_model.h5"   # Path to the saved model
model = load_model(model_path)

# Define the image path
img_path = "/content/pathhole_dataset/potholes/10.jpg"  # Path to your image

# Define the dataset directory
dataset_dir = "/content/pathhole_dataset"

# Extract class labels from subdirectories within the dataset directory
class_labels = sorted(os.listdir(dataset_dir))

# Preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

# Make predictions using your model
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions)
confidence_score = np.max(predictions)
predicted_class = class_labels[predicted_class_index]

# Check for confusion between predicted class and dataset classes
if predicted_class not in class_labels:
    # Get predictions from ImageNet model
    imagenet_predictions = decode_predictions(predictions, top=5)[0]

    # Check if any of the ImageNet predictions match dataset classes
    for label in imagenet_predictions:
        if label[1] in class_labels:
            predicted_class = label[1]
            confidence_score = label[2]
            break

# Print the final predicted class and confidence score
print("Predicted class:", predicted_class)
print("Confidence score:", confidence_score)


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing import image
from google.colab.patches import cv2_imshow  # Use in Colab to display images

# Load trained DenseNet201 model
model_path = "/content/DenseNet201_model.h5"
model = tf.keras.models.load_model(model_path, compile=False)  # Fix Warning

# Define class labels
class_labels = ["Normal Road", "Path Hole"]  # Adjust based on dataset classes

# Upload your video
video_path = "/content/traveling whatsapp status, nature whatsapp status-tamil,travel status, nature status,status status.mp4"
cap = cv2.VideoCapture(video_path)

# Get video properties
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define output video writer
output_path = "/content/detected_video2.avi"
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Exit loop if no frame

    # Preprocess frame for model
    img = cv2.resize(frame, (224, 224))  # Resize for DenseNet201
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)

    # Predict class
    predictions = model.predict(img)
    class_index = np.argmax(predictions)
    label = class_labels[class_index]
    confidence = predictions[0][class_index] * 100

    # Draw bounding box and label
    color = (0, 255, 0) if label == "Normal Road" else (0, 0, 255)
    cv2.rectangle(frame, (50, 50), (frame_width - 50, frame_height - 50), color, 3)
    text = f"{label}: {confidence:.2f}%"
    cv2.putText(frame, text, (60, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # Save processed frame to output video
    out.write(frame)

    # Show video frame in Colab
    cv2_imshow(frame)  # Fixed for Colab

# Release resources
cap.release()
out.release()

print(f" Detection Complete! Download your video: {output_path}")
